In [5]:
from sql_functions import get_dataframe
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

import warnings
warnings.filterwarnings("ignore")
RSEED = 10

In [6]:
schema = "bgg_data"
kickstarter = "kickstarter_unique_campaigns"
slug = "unique_slug_bgg_id"
main_stats = "unfiltered_main_stats_cleaned"
stat = "statistics"

df_ks = get_dataframe(f"SELECT * FROM {schema}.{kickstarter}")
df_slug = get_dataframe(f"SELECT * FROM {schema}.{slug}")
df_main_stats = get_dataframe(f"SELECT * FROM {schema}.{main_stats}")
#df_ks.rename({'bgg_id':'id'},axis=1,inplace=True)

In [7]:
df_ks_slug = pd.merge(df_slug,df_ks,on='slug')
df_ks_slug.rename({'bgg_id':'id'},axis=1,inplace=True)

In [8]:
df = pd.merge(df_ks_slug,df_main_stats,on='id')

In [9]:
df.columns

Index(['slug', 'id', 'deadline', 'created_at', 'launched_at', 'country',
       'currency', 'goal', 'pledged', 'backers_count', 'usd_pledged',
       'state_changed_at', 'successful', 'yearpublished', 'min_players',
       'max_players', 'playtime', 'min_playtime', 'max_playtime', 'min_age',
       'average', 'user_rated', 'num_owned', 'trading', 'wanting', 'wishing',
       'numcomments', 'numweights', 'averageweight'],
      dtype='object')

In [10]:
cols_to_drop = ['id',
                'slug',
                'deadline', 
                'created_at',
                'launched_at',
                'country',
                'pledged',
                'backers_count',
                'currency',
                'state_changed_at',
                'successful',
                'yearpublished',
                'playtime',
                'wishing',
                'numcomments',
                'user_rated',
                'num_owned',
                'trading',
                'numweights']

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4537 entries, 0 to 4536
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   slug              4537 non-null   object        
 1   id                4537 non-null   int64         
 2   deadline          4537 non-null   datetime64[ns]
 3   created_at        4537 non-null   datetime64[ns]
 4   launched_at       4537 non-null   datetime64[ns]
 5   country           4537 non-null   object        
 6   currency          4537 non-null   object        
 7   goal              4537 non-null   int64         
 8   pledged           4537 non-null   float64       
 9   backers_count     4537 non-null   int64         
 10  usd_pledged       4537 non-null   float64       
 11  state_changed_at  4537 non-null   datetime64[ns]
 12  successful        4537 non-null   bool          
 13  yearpublished     4405 non-null   float64       
 14  min_players       4521 n

In [95]:
df.drop(cols_to_drop,axis=1,inplace=True)

In [96]:
df.dropna(axis=0,inplace=True)

In [97]:
stats.zscore(df['min_playtime'])

2      -0.246679
3      -0.246679
4       0.012582
6       0.401475
7       0.401475
          ...   
4530   -0.117049
4531    0.142213
4532    0.012582
4533   -0.203469
4535   -0.289890
Name: min_playtime, Length: 2950, dtype: float64

In [98]:
df = df[(np.abs(stats.zscore(df['min_playtime'])) < 2)]
df = df[(np.abs(stats.zscore(df['max_playtime'])) < 2)]
df = df[(np.abs(stats.zscore(df['max_players'])) < 2)]
df = df[(np.abs(stats.zscore(df['min_players'])) < 3)]
df = df[df["min_age"] < 19]

In [99]:
df.describe()

,goal,usd_pledged,min_players,max_players,min_playtime,max_playtime,min_age,average,wanting,averageweight
count,2.810000e+03,2.810000e+03,2810.00000,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000,2810.000000
mean,2.530447e+04,1.607831e+05,1.73879,4.673665,40.556940,70.376157,11.080427,7.016100,46.670107,2.196895
std,6.616277e+04,5.760943e+05,0.61147,1.687361,29.522215,50.316731,3.195194,1.156285,124.743347,0.767368
min,5.000000e+00,0.000000e+00,1.00000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,5.000000e+03,9.106225e+03,1.00000,4.000000,20.000000,30.000000,9.000000,6.466095,2.000000,1.666700
50%,1.200000e+04,2.656650e+04,2.00000,4.000000,30.000000,60.000000,12.000000,7.066180,9.000000,2.000000
75%,2.500000e+04,8.984550e+04,2.00000,6.000000,60.000000,90.000000,14.000000,7.673322,36.000000,2.750000
max,2.200000e+06,1.296961e+07,3.00000,16.000000,250.000000,360.000000,18.000000,10.000000,2008.000000,4.817300


In [100]:
y = df["usd_pledged"]
X = df.drop("usd_pledged", axis=1)

In [102]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2810 entries, 2 to 4535
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   goal           2810 non-null   int64  
 1   min_players    2810 non-null   float64
 2   max_players    2810 non-null   float64
 3   min_playtime   2810 non-null   float64
 4   max_playtime   2810 non-null   float64
 5   min_age        2810 non-null   int64  
 6   average        2810 non-null   float64
 7   wanting        2810 non-null   int64  
 8   averageweight  2810 non-null   float64
dtypes: float64(6), int64(3)
memory usage: 219.5 KB


In [108]:
# Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [109]:
classifier = LogisticRegression() # instantiate a sklearn logistic regression class
classifier.fit(X_train, y_train) # fit the classifier/model on our train data 
y_prediction = classifier.predict(X_test) # use the fit model to predict on our test data 

#have a look at the predicitons
#y_prediction[:10]

ValueError: Unknown label type: 'continuous'